# Optunaを用いたクッキーレシピ最適化

オープンソースのハイパーパラメータ最適化フレームワーク[Optuna](https://optuna.org)を使って、チョコチップクッキーのレシピを最適化するためのノートブックです。

実験では下記のように、繰り返しクッキーを作り、試食をして、徐々によいレシピを見つけていきます。１日１レシピを試すなど、気長に実験をしていきましょう。

1. Optunaによるレシピの提案
2. レシピに基づいてクッキーを焼く
3. クッキーの評価値をOptunaに報告
4. 1に戻る

最初、Optunaはあなたの好みについて何も知りませんから、とんでもないレシピ(例: バターも砂糖もチョコチップも上限いっぱいのクッキーや、逆に下限すれすれのクッキー)を推薦することがあります。そのような時、我慢して試作していては、つらくて実験をやめたくなってしまうかもしれません。代わりに、あなた自身でレシピを修正し、登録しなおせるようになっていますので、ご安心ください。

最適化の手順や探索空間などは、Googleの研究者による論文[Bayesian Optimization for a Better Dessert](https://static.googleusercontent.com/media/research.google.com/ja//pubs/archive/46507.pdf)に基づいています。論文ではピッツバーグとカリフォルニアの２か所で実験を行い、それぞれのベストレシピを報告しています。このノートブックにも、レシピを載せています。実験の最初に、それらを作ってみて、スコアをOptunaに報告しておくと、より早く、好みのレシピにたどり着けるかもしれません。ノートブックの最後にそのためのセルを用意していますので、ご利用ください。

なお、下記の理由で論文と完全に一致するものではないことをご了承ください。

- 最適化ソフトウェアが論文ではGoogle Vizierを使っているのに対し、このノートブックではOptunaを使っている。
- 論文に書かれていない情報がある (例: 離散変数のステップ幅)
- 論文ではシェフのフィードバックをもとに探索空間を変えていた

最後の点について、あなたも探索範囲を変えることを試してみると良いかもしれません。実験がアメリカで行われたためか、バター、砂糖、チョコチップの量は、最小にしたとしても、日本でよくみるレシピより多くなっているように思います。

では、あなたにとって、よいクッキーが見つかりますように！

## 0. 環境構築

Optunaをインストールし、探索空間とユーティリティ関数を読み込みます。

In [1]:
%pip install --progress-bar off --quiet optuna pandas plotly

Note: you may need to restart the kernel to use updated packages.


In [1]:
import optuna
optuna.__version__

'3.0.3'

探索空間の初期値は、カリフォルニアでの実験の探索空間をベースに、不足している情報(例、小さじのきざみ幅など)を補ったものです。あなたの好みに合わせて、探索空間を変えることができます。
変更した場合には、ノートブックを保存することを忘れないでください！


In [2]:
# チョコチップの種類
chocolate_chip_types = ["dark", "milk"]

# チョコチップの量。単位はグラム。
chocolate_chip_low = 84
chocolate_chip_high = 260

# ベーキングパウダーの量。単位は小さじ。1/16きざみ
# 本来は 0,0548 から 0.655 だが、0.0548はスプレッドシートのエラーが原因とのこと
# 計量の精度の限界も踏まえて1/16 (0.0625) から 11/16 (0.6875) で代替
baking_soda_low = 1 / 16
baking_soda_high = 11 / 16
baking_soda_step = 1 / 16

# バターの量。単位はグラム。
butter_low = 45
butter_high = 175

# たまごの量。単位はグラム。
egg_low = 25
egg_high = 35

# 食塩の量。単位は小さじ。1/8きざみ
salt_low = 1 / 4
salt_high = 1
salt_step = 1 / 8

# 砂糖の総量。単位はグラム
sugar_low = 90
sugar_high = 500

# ブラウンシュガーの割合。単位は%。残りは白砂糖。
brown_sugar_low = 0
brown_sugar_high = 1

# バニラエッセンスの量。単位は小さじ。 1/8きざみ
vanilla_low = 0
vanilla_high = 1.25
vanilla_step = 1 / 8

# オレンジエッセンスの量。単位は小さじ。1/8きざみ
orange_low = 0
orange_high = 3 / 4
orange_step = 1 / 8

# @markdown カイエンペッパーの量。単位は小さじ。1/8きざみ
cayenne_pepper_low = 0
cayenne_pepper_high = 1 / 2
cayenne_pepper_step = 1 / 8

In [3]:
# 入力された範囲をもとに、Optunaの探索空間を定義します
from optuna.distributions import (
    CategoricalDistribution, 
    FloatDistribution,
    IntDistribution,
)

from IPython.display import Markdown

def define_search_space():
    return {
        "chocolate_chip_type": CategoricalDistribution(choices=chocolate_chip_types),
        "chocolate_chip": IntDistribution(low=chocolate_chip_low, high=chocolate_chip_high),
        "baking_soda": FloatDistribution(low=baking_soda_low, high=baking_soda_high, step=baking_soda_step),
        "butter": IntDistribution(low=butter_low, high=butter_high),
        "egg": IntDistribution(low=egg_low, high=egg_high),
        "salt": FloatDistribution(low=salt_low, high=salt_high, step=salt_step),
        "sugar": IntDistribution(low=sugar_low, high=sugar_high),
        "brown_sugar": FloatDistribution(low=brown_sugar_low, high=brown_sugar_high),
        "vanilla": FloatDistribution(low=vanilla_low, high=vanilla_high, step=vanilla_step),
        "orange": FloatDistribution(low=orange_low, high=orange_high, step=orange_step),
        "cayenne_pepper": FloatDistribution(low=cayenne_pepper_low, high=cayenne_pepper_high, step=cayenne_pepper_step),
    }


# ユーティリティ関数
# クッキーのレシピを表形式で表示
ingredients = {
    "chocolate_chip": ("チョコチップ", "グラム"),
    "baking_soda": ("ベーキングパウダー", "小さじ"),
    "salt": ("食塩", "小さじ"),
    "cayenne_pepper": ("唐辛子", "小さじ"),
    "sugar": ("砂糖", "グラム"),
    "egg": ("たまご", "グラム"),
    "butter": ("バター", "グラム"),
    "orange": ("オレンジエッセンス", "小さじ"),
    "vanilla": ("バニラエッセンス", "小さじ"),
}

def create_recipe_table(params):
    line = "| 材料 | 単位 | 種類・量 |\n"
    line += "| --- | --- | --- |\n"
    # 小麦粉
    value = 167
    line += f"| 小麦粉 | グラム | {value:.1f} |\n"
    # チョコチップの種類
    line += f"| チョコチップの種類 | N/A | {params['chocolate_chip_type']} |\n" 
    for name in ingredients.keys():
        value = params[name]
        if name == "brown_sugar":
            # see "sugar" branch for usage of "brown_sugar"
            continue
        if name == "sugar":
            sugar_total = params["sugar"]
            value = params["brown_sugar"] * sugar_total
            line += f"| ブラウンシュガー | グラム | {value:.1f} |\n"
            value = sugar_total - value
            line += f"| 白砂糖 | グラム | {value:.1f} |\n"
            continue
        _name, _unit = ingredients[name]
        value = params[name]
        line += f"| {_name} | {_unit} | {value:.2f} |\n"
    return line


## 1. Optunaによるレシピの提案

In [4]:
study = optuna.create_study(storage="sqlite:///bayesian_cookie.db", study_name="bayesian-cookie-1", load_if_exists=True, direction="maximize")

[I 2022-11-08 03:09:58,535] A new study created in RDB with name: bayesian-cookie-1


In [5]:
search_space = define_search_space()
trial = study.ask(fixed_distributions=search_space)

In [6]:
trial.params

{'chocolate_chip_type': 'dark',
 'chocolate_chip': 214,
 'baking_soda': 0.6875,
 'butter': 167,
 'egg': 31,
 'salt': 0.875,
 'sugar': 304,
 'brown_sugar': 0.5551878664330968,
 'vanilla': 0.625,
 'orange': 0.0,
 'cayenne_pepper': 0.0}

In [7]:
# 読みやすいように表形式でレシピを表示します
Markdown(create_recipe_table(trial.params))

| 材料 | 単位 | 種類・量 |
| --- | --- | --- |
| 小麦粉 | グラム | 167.0 |
| チョコチップの種類 | N/A | dark |
| チョコチップ | グラム | 214.00 |
| ベーキングパウダー | 小さじ | 0.69 |
| 食塩 | 小さじ | 0.88 |
| 唐辛子 | 小さじ | 0.00 |
| ブラウンシュガー | グラム | 168.8 |
| 白砂糖 | グラム | 135.2 |
| たまご | グラム | 31.00 |
| バター | グラム | 167.00 |
| オレンジエッセンス | 小さじ | 0.00 |
| バニラエッセンス | 小さじ | 0.62 |


In [8]:
# このレシピに対する評価値を報告する際に、`trial.number` の値が必要になります。レシピと一緒にメモしておいてください。
trial.number

0

### (オプショナル) 推薦されたレシピを修正する場合

In [9]:
# suggestされたparamsを編集する場合には、上記の `trial.params` の値を修正して、 `modified_params` という辞書を作ります。
modified_params = {
 'baking_soda': 0.625,
 'brown_sugar': 0.7280600317739677,
 'butter': 83,
 'cayenne_pepper': 0.375,
 'chocolate_chip': 154,
 'chocolate_chip_type': 'dark',
 'egg': 26,
 'orange': 0.25,
 'salt': 0.375,
 'sugar': 395,
 'vanilla': 0.375   
}

In [10]:
# 修正したparamsをOptunaに登録しなおします。
study.tell(trial, state=optuna.trial.TrialState.FAIL)
study.add_trial(
    optuna.create_trial(
        state=optuna.trial.TrialState.WAITING,
        params=modified_params,
        distributions=search_space,
        user_attrs={"manual": True}
    )
)
trial = study.ask(fixed_distributions=search_space)

In [11]:
# 修正したparamsを確認します。
Markdown(create_recipe_table(trial.params))

| 材料 | 単位 | 種類・量 |
| --- | --- | --- |
| 小麦粉 | グラム | 167.0 |
| チョコチップの種類 | N/A | dark |
| チョコチップ | グラム | 154.00 |
| ベーキングパウダー | 小さじ | 0.62 |
| 食塩 | 小さじ | 0.38 |
| 唐辛子 | 小さじ | 0.38 |
| ブラウンシュガー | グラム | 287.6 |
| 白砂糖 | グラム | 107.4 |
| たまご | グラム | 26.00 |
| バター | グラム | 83.00 |
| オレンジエッセンス | 小さじ | 0.25 |
| バニラエッセンス | 小さじ | 0.38 |


## 2. レシピに基づいてクッキーを焼く

Optunaで推薦されたレシピに基づいて、クッキーを焼きましょう。
推薦された材料のほかに、小麦粉167gが必要になります。

### 生地の作り方

1. 小麦粉、ベーキングパウダー、カイエンペッパー、食塩を混ぜ合わせます
2. （冷蔵庫から取り出したばかりの）バターとたまご、バニラエッセンス、オレンジエッセンスを、クリーム状になるまで(約２分間)ミキサーで混ぜます
3. 1と2を均一になるまで混ぜ合わせます。混ぜすぎないように注意。
4. (必要に応じて)生地を冷蔵庫で休ませます
5. スプーンですくって、オーブン用ペーパーの上に並べたら生地は出来上がりです

### 焼き方

生地をオーブンで焼きましょう。オーブンの温度と時間は175度で14分が基準になっています。
ただし、オーブンはメーカーや機種によって焼き上がりが異なります。
ご自分の家のオーブンに合わせて温度や時間を調整してください。

In [12]:
Markdown(create_recipe_table(trial.params))

| 材料 | 単位 | 種類・量 |
| --- | --- | --- |
| 小麦粉 | グラム | 167.0 |
| チョコチップの種類 | N/A | dark |
| チョコチップ | グラム | 154.00 |
| ベーキングパウダー | 小さじ | 0.62 |
| 食塩 | 小さじ | 0.38 |
| 唐辛子 | 小さじ | 0.38 |
| ブラウンシュガー | グラム | 287.6 |
| 白砂糖 | グラム | 107.4 |
| たまご | グラム | 26.00 |
| バター | グラム | 83.00 |
| オレンジエッセンス | 小さじ | 0.25 |
| バニラエッセンス | 小さじ | 0.38 |


## 3. クッキーの評価値をOptunaに報告

評価基準は下記です。点が大きいほど、よい評価になっています。

1.   まずい。二度と作らないでほしい
2.   美味しくない。もっと美味しいクッキーがたくさんある
3.   まずまずだけど、印象的ではない
4.   おいしいクッキーだけど、特筆すべきものではない
5.   とても美味しいクッキー
6.   すばらしい。群を抜いている
7.   これまでに食べた中で最高に近いチョコチップクッキー

レシピの評価値を次のセルの `objective_value` に入力して登録しましょう。


In [13]:
# objective_valueの値を1から7で指定してください
objective_value = 4

In [14]:
# このセルを実行すると、評価値がoptunaに報告されます。
study.tell(trial, values=objective_value)

FrozenTrial(number=1, values=[4.0], datetime_start=datetime.datetime(2022, 11, 8, 3, 10, 2, 950938), datetime_complete=datetime.datetime(2022, 11, 8, 3, 10, 9, 11841), params={'baking_soda': 0.625, 'brown_sugar': 0.7280600317739677, 'butter': 83, 'cayenne_pepper': 0.375, 'chocolate_chip': 154, 'chocolate_chip_type': 'dark', 'egg': 26, 'orange': 0.25, 'salt': 0.375, 'sugar': 395, 'vanilla': 0.375}, distributions={'baking_soda': FloatDistribution(high=0.6875, log=False, low=0.0625, step=0.0625), 'brown_sugar': FloatDistribution(high=1.0, log=False, low=0.0, step=None), 'butter': IntDistribution(high=175, log=False, low=45, step=1), 'cayenne_pepper': FloatDistribution(high=0.5, log=False, low=0.0, step=0.125), 'chocolate_chip': IntDistribution(high=260, log=False, low=84, step=1), 'chocolate_chip_type': CategoricalDistribution(choices=('dark', 'milk')), 'egg': IntDistribution(high=35, log=False, low=25, step=1), 'orange': FloatDistribution(high=0.75, log=False, low=0.0, step=0.125), 'salt

In [15]:
# もし `trial_number` を忘れてしまった場合には、下記の一覧表から該当するレシピを探しましょう
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_baking_soda,params_brown_sugar,params_butter,params_cayenne_pepper,params_chocolate_chip,params_chocolate_chip_type,params_egg,params_orange,params_salt,params_sugar,params_vanilla,user_attrs_manual,state
0,0,NaN,2022-11-08 03:09:58.821597,2022-11-08 03:10:02.906185,0 days 00:00:04.084588,0.6875,0.555188,167,0.000,214,dark,31,0.00,0.875,304,0.625,NaN,FAIL
1,1,4.0,2022-11-08 03:10:02.950938,2022-11-08 03:10:09.011841,0 days 00:00:06.060903,0.6250,0.728060,83,0.375,154,dark,26,0.25,0.375,395,0.375,True,COMPLETE


## 4. ステップ1. Optunaによるレシピの提案に戻る

以上で1つのクッキーレシピの提案、試作、評価が終了しました。
ステップ1のOptunaによるレシピの提案に戻って新しいクッキーレシピを試しましょう。

## 付録: 既存のレシピとその評価値の入力

Bayesian Cookieの元論文には、複数のレシピが掲載されています。
以下では各レシピを表示し、レシピに対する評価値を追加できるようにしました。

### コントロールクッキー

In [18]:
control_cookie_params = dict(
    # チョコチップの種類
    chocolate_chip_type = "milk",

    # チョコチップの量。単位はグラム。
    chocolate_chip = 160,

    # ベーキングパウダーの量。単位は小さじ。1/8きざみ
    baking_soda = 1 / 2,

    # バターの量。単位はグラム。
    butter = 125,

    # たまごの量。単位はグラム。
    egg = 30,

    # 食塩の量。単位は小さじ。1/8きざみ
    salt = 1 / 4,

    # 砂糖の総量。単位はグラム
    sugar = 300,

    # ブラウンシュガーの割合。単位は%。残りは白砂糖。
    brown_sugar = 0.5,

    # バニラエッセンスの量。単位は小さじ。 1/8きざみ
    vanilla = 1 / 2,

    # オレンジエッセンスの量。単位は小さじ。1/8きざみ
    orange = 0,

    # カイエンペッパーの量。単位は小さじ。1/8きざみ
    cayenne_pepper = 0,
)
Markdown( create_recipe_table(control_cookie_params))

| 材料 | 単位 | 種類・量 |
| --- | --- | --- |
| 小麦粉 | グラム | 167.0 |
| チョコチップの種類 | N/A | milk |
| チョコチップ | グラム | 160.00 |
| ベーキングパウダー | 小さじ | 0.50 |
| 食塩 | 小さじ | 0.25 |
| 唐辛子 | 小さじ | 0.00 |
| ブラウンシュガー | グラム | 150.0 |
| 白砂糖 | グラム | 150.0 |
| たまご | グラム | 30.00 |
| バター | グラム | 125.00 |
| オレンジエッセンス | 小さじ | 0.00 |
| バニラエッセンス | 小さじ | 0.50 |


In [19]:
# コントロールクッキーの評価値を1から7で指定
control_cookie_objective_value = 4

In [20]:
# このセルを実行するとコントロールクッキーの評価値が登録されます
study.add_trial(
    optuna.create_trial(
        state=optuna.trial.TrialState.COMPLETE,
        params=control_cookie_params,
        distributions=search_space,
        user_attrs={"recipe": "control"},
        value=control_cookie_objective_value
    )
)

### ピッツバーグクッキー

In [21]:
pittsburgh_cookie_params = dict(
    # ョコチップの種類
    chocolate_chip_type = "dark",

    # チョコチップの量。単位はグラム。
    chocolate_chip = 196,

    # ベーキングパウダーの量。単位は小さじ。1/8きざみ
    baking_soda = 1 / 2,

    # バターの量。単位はグラム。
    butter = 129,

    # たまごの量。単位はグラム。
    egg = 30,

    # 食塩の量。単位は小さじ。1/8きざみ
    salt = 1 / 4,

    # 砂糖の総量。単位はグラム
    sugar = 108,

    # ブラウンシュガーの割合。単位は%。残りは白砂糖。
    brown_sugar = 0.88,

    # バニラエッセンスの量。単位は小さじ。 1/8きざみ
    vanilla = 1 / 2,

    # オレンジエッセンスの量。単位は小さじ。1/8きざみ
    orange = 3 / 8,

    # カイエンペッパーの量。単位は小さじ。1/8きざみ
    cayenne_pepper = 1 / 4
)
Markdown( create_recipe_table(pittsburgh_cookie_params))

| 材料 | 単位 | 種類・量 |
| --- | --- | --- |
| 小麦粉 | グラム | 167.0 |
| チョコチップの種類 | N/A | dark |
| チョコチップ | グラム | 196.00 |
| ベーキングパウダー | 小さじ | 0.50 |
| 食塩 | 小さじ | 0.25 |
| 唐辛子 | 小さじ | 0.25 |
| ブラウンシュガー | グラム | 95.0 |
| 白砂糖 | グラム | 13.0 |
| たまご | グラム | 30.00 |
| バター | グラム | 129.00 |
| オレンジエッセンス | 小さじ | 0.38 |
| バニラエッセンス | 小さじ | 0.50 |


In [22]:
# ピッツバーグクッキーの評価値を1から7で指定
pittsburgh_cookie_objective_value = 6

In [23]:
study.add_trial(
    optuna.create_trial(
        state=optuna.trial.TrialState.COMPLETE,
        params=pittsburgh_cookie_params,
        distributions=search_space,
        user_attrs={"recipe": "pittsburgh"},
        value=pittsburgh_cookie_objective_value
    )
)

### カリフォルニアクッキー

In [24]:
# たまごとバターの量は、小数点以下を四捨五入してあります。
# 正確にはたまご25.7g、バター81.3gです。

california_cookie_params = dict(

    # チョコチップの種類
    chocolate_chip_type = "milk",

    # チョコチップの量。単位はグラム。
    chocolate_chip = 245,

    # ベーキングパウダーの量。単位は小さじ。1/8きざみ
    baking_soda = 5 / 8,

    # バターの量。単位はグラム。
    butter = 81,

    # たまごの量。単位はグラム。
    egg = 26,

    # 食塩の量。単位は小さじ。1/8きざみ
    salt = 1 / 2,

    # 砂糖の総量。単位はグラム
    sugar = 127,

    # ブラウンシュガーの割合。単位は%。残りは白砂糖。
    brown_sugar = 0.31,

    # バニラエッセンスの量。単位は小さじ。 1/8きざみ
    vanilla = 3 / 4,

    # オレンジエッセンスの量。単位は小さじ。1/8きざみ
    orange = 1 / 8,

    # カイエンペッパーの量。単位は小さじ。1/8きざみ
    cayenne_pepper = 1 / 8,
)
Markdown(create_recipe_table(california_cookie_params))

| 材料 | 単位 | 種類・量 |
| --- | --- | --- |
| 小麦粉 | グラム | 167.0 |
| チョコチップの種類 | N/A | milk |
| チョコチップ | グラム | 245.00 |
| ベーキングパウダー | 小さじ | 0.62 |
| 食塩 | 小さじ | 0.50 |
| 唐辛子 | 小さじ | 0.12 |
| ブラウンシュガー | グラム | 39.4 |
| 白砂糖 | グラム | 87.6 |
| たまご | グラム | 26.00 |
| バター | グラム | 81.00 |
| オレンジエッセンス | 小さじ | 0.12 |
| バニラエッセンス | 小さじ | 0.75 |


In [25]:
# カリフォルニアクッキーの評価値を1から7で指定
california_cookie_objective_value = 6

In [26]:
study.add_trial(
    optuna.create_trial(
        state=optuna.trial.TrialState.COMPLETE,
        params=california_cookie_params,
        distributions=search_space,
        user_attrs={"recipe": "california"},
        value=california_cookie_objective_value,
    )
)

In [27]:
# 結果の確認
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_baking_soda,params_brown_sugar,params_butter,params_cayenne_pepper,params_chocolate_chip,params_chocolate_chip_type,params_egg,params_orange,params_salt,params_sugar,params_vanilla,user_attrs_manual,user_attrs_recipe,state
0,0,NaN,2022-11-08 03:09:58.821597,2022-11-08 03:10:02.906185,0 days 00:00:04.084588,0.6875,0.555188,167,0.000,214,dark,31,0.000,0.875,304,0.625,NaN,NaN,FAIL
1,1,4.0,2022-11-08 03:10:02.950938,2022-11-08 03:10:09.011841,0 days 00:00:06.060903,0.6250,0.728060,83,0.375,154,dark,26,0.250,0.375,395,0.375,True,NaN,COMPLETE
2,2,4.0,2022-11-08 03:10:21.752748,2022-11-08 03:10:21.752748,0 days 00:00:00,0.5000,0.500000,125,0.000,160,milk,30,0.000,0.250,300,0.500,NaN,control,COMPLETE
3,3,6.0,2022-11-08 03:10:24.023883,2022-11-08 03:10:24.023883,0 days 00:00:00,0.5000,0.880000,129,0.250,196,dark,30,0.375,0.250,108,0.500,NaN,pittsburgh,COMPLETE
4,4,6.0,2022-11-08 03:10:26.775965,2022-11-08 03:10:26.775965,0 days 00:00:00,0.6250,0.310000,81,0.125,245,milk,26,0.125,0.500,127,0.750,NaN,california,COMPLETE


## 付録: 結果の可視化

In [28]:
optuna.visualization.plot_optimization_history(study)

In [29]:
optuna.visualization.plot_parallel_coordinate(study)